In [33]:
hateful = loadfile("util/hateful.lua")()
return hateful

{config = <function 1>,deserialize = <function 2>,read = <function 3>,serialize = <function 4>,string_deserialize = <function 5>,write = <function 6>}

In [34]:
o = hateful.config("/tmp/test2.conf")
save = true
o.set("x", 1, save)
o.set("y", 1, save)
o.set("z", {x = 1, y = {1 + 2}}, save)
o.set("t", 1, save)
return o.get("x"), o.cache

1	{t = 1,x = 1,y = 1,z = {x = 1,y = { 3 }}}

In [35]:
o2 = hateful.config("/tmp/test2.conf")
return o2

{default = {},file = "/tmp/test2.conf",get = <function 1>,save = <function 2>,set = <function 3>,<metatable> = {__index = <function 4>}}

In [26]:
t = {
    rice = "Openbox-based",
    scheme = "Fleon",
    params = {
        terminal = "urxvt",
        browser = "firefox",
        arg = {
            x = "asdad",
            y = 1,
        },
        ntags = 5
    },
    123,
    [true] = 1
}

In [36]:
o2.cache

{t = 1,x = 1,y = 1,z = {x = 1,y = { 3 }}}

In [124]:
s = hateful.serialize(t)
print(s)
print(("="):rep(30))
return hateful.string_deserialize(s) 

1 = 123
rice = Openbox-based
params = {
	browser = firefox
	terminal = urxvt
	ntags = 5
	arg = {
		x = asdad
		y = 1
	}
}
scheme = Fleon
true = 1


{ 123,[true] = 1,params = {arg = {x = "asdad",y = 1},browser = "firefox",ntags = 5,terminal = "urxvt"},rice = "Openbox-based",scheme = "Fleon"}

In [37]:
hateful.deserialize = function(s)
    local obj = {}
    local state =  {
        key = "",
        val = "",
        recursion = "",
        counting = false,
    }
    s = s:gsub("%s*=%s*", "="):gsub("^%s*", ""):gsub("%s*$", "")
    local mode = "key"
    for ch in s:gmatch('.') do
        if ch == " " or ch == "\t" then 
            if state.counting then
                state[mode] = state[mode] .. ch
            end
        elseif ch == "=" then
            state.counting = false
            mode = "val"
        elseif ch == "{" then
            mode = "recursion"
        elseif ch == "}" then
            mode = "key"
            obj[state.key] = hateful.deserialize(state.recursion)
            print(state.key, hateful.deserialize(state.recursion))
            state.key = ""
            state.val = ""
            state.counting = false
        elseif ch == "\n" then
            -- print("newline", state.key, "=", state.val)
            if mode ~= "recursion" then
                obj[state.key] = state.val
                state.counting = false
                state.key = ""
                state.val = ""
                mode = "key"
            end
         else
            if state.counting == false then
                state.counting = true
            end
            state[mode] = state[mode] .. ch
        end
    end
    return obj
end

return hateful.deserialize(s)

	table: 0x562c9914df80
	table: 0x562c99191610


{[""] = "",["1"] = "123",arg = "asdad",ntags = "5",params = "firefox",rice = "Openbox-based",scheme = "Fleon",terminal = "urxvt",y = "1"}

In [111]:
parsetype = function(v)
    if type(v) ~= "string" then return v end
    if v:match("%s*nil%s*") then return v end
    if v:match("%s*(%d+)%s*") then return tonumber(v) end
    if v:match("%s*(true)%s*") then return true end
    if v:match("%s*(false)%s*") then return false end
    return v
end
hateful.deserialize = function(s)
    local obj = {}
    local r = 0
    local rs, key, val = "", "", ""
    s = s:gsub("%s*=%s*", "="):gsub("^%s*", ""):gsub("%s*$", "")
    for line in s:gmatch('[^\n]+') do
        if line:find("{") then
            if r == 0 then
                key = line:match("^%s*([a-zA-Z0-9_- ]+)%s*=")
            else
                rs = rs .. line .. "\n"
            end
            r = r + 1
        elseif line:find("}") then
            r = r - 1
            if r == 0 then
                obj[key] = hateful.deserialize(rs)
                rs = ""
            else
                rs = rs .. line .. "\n"
            end
        elseif key and val then
            if r == 0 then
                key, val = line:match("%s*([a-zA-Z0-9_- ]+)%s*=%s*([{}a-zA-Z0-9_ -]+)%s*$")
                key, val = parsetype(key), parsetype(val)
                obj[key] = val
            else
                rs = rs .. line .. "\n"
            end
        else error("Error on line", line)
        end
    end
    return obj
end

return hateful.deserialize(s)

{ 123,[true] = 1,params = {arg = {x = "asdad",y = 1},browser = "firefox",ntags = 5,terminal = "urxvt"},rice = "Openbox-based",scheme = "Fleon"}

In [103]:
t

{ 123,[true] = 1,params = {arg = {x = "asdad",y = 1},browser = "firefox",ntags = 5,terminal = "urxvt"},rice = "Openbox-based",scheme = "Fleon"}

In [16]:
s_serialize = function(t, il)
    local s = ""
    local entrycount = 0
    if not il then il = 0 end --[[ indent level ]]
    local indent = string.rep('\t', il)
    for k, v in pairs(t) do
        if entrycount == 0 then
            entrycount = entrycount + 1
        else
            s = s .. indent
        end
        if type(v) == "table" then
            s = s .. "(" .. tostring(k) .. ", "
            s = s .. s_serialize(v, il + 1) .. ")\n"
        else
            s = s .. "(" .. tostring(k) .. ", " .. tostring(v) .. "),\n"
        end
    end
    return "[" .. s:sub(1, #s-2) .. "]"
end
s_deserialize = function(s)
    local state = {
        key = "",
        iskey = true,
        isarray = 0,
        str = "",
        val = "",
        obj = {},
    }
    for ch in s:gmatch('.') do
        if ch:match('[a-zA-Z0-9-]') then
            if state.iskey then
                state.key = state.key .. ch
            else
                state.val = state.val .. ch
            end
        elseif ch == ',' then
            state.iskey = not state.iskey
        elseif ch == "[" then
            state.isarray = state.isarray + 1
        elseif ch == "]" then
            state.isarray = state.isarray - 1
        end
    end
    assert(state.isarray == 0, "] expected")
    return state.obj
end

In [17]:
ss = s_serialize(t)
print(ss)
print(string.rep('-', 30))
st = s_deserialize(ss)
return st

[(1, 123),
(rice, Openbox-based),
(params, [(browser, firefox),
	(terminal, urxvt),
	(ntags, 5),
	(arg, [(x, asdad),
		(y, 1)]])
(scheme, Fleon),
(true, 1)]
------------------------------


{}

- Success: I have been (...) (what you good at)
- Strength: My real strength is (...)
- Situation: How that strength apply to the situation  "What I am look  for is"
- BTVN: Tell me about your self, introduce yourself, 

In [93]:
return 1

1